(onboardone)=
# Import a World Bank Eviews model from its solution file (.wf1)
This notebook takes a .wf1 workfile and transforms it into a modelflow model.

Most standard World Bank models should work with limited intervention.  Some using unusual techniques or variable definitions may require intervention by the user.

## Overview of the import process

The overall import process is performed by a special a ModelFlow class named:**GrabWfModel**. Certain steps require the use of EViews itself.  This howoto was designed usinng EViews version 12, but and has been tested with versions 13 and 14.

Steps to follow:
 1. Start EViews and open the solution file 
    -    If returning to this step after an initial error perform transformations on the data (needed in some cases where special EViews functions are used in the model).
    -    The UNlink the model. This transforms linked equations into explicit equations in the EViews model object. 
    -    Save the revised model and data as a .wf2 file. (the .wf2 is a JSON format that can be easily read into python).
    **QCVD?  Same name with ```_modelflow appended```**
 5. Close Eviews.
 6. The wf2 file is read as a json file. 
 7. Relevant objects are extracted. 
 7. The MFMSA variable is  extracted, to be saved in the dumpfile. 
 8. The equations are transformed and normalized to modelflow format and classified into identities and stochastic
 9. Stochastic equations are enriched by add_factor and fixing terms (dummy + fixing value)  
 9. For Stochastic equations new fitted variables are generated - without add add_factors and dummies.  
 9. A model to generate fitted variables is created  
 9. A model to generate add_factors is created. 
 9. A model encompassing the original equations, the model for fitted variables and for add_factors is created. 
 9. The data series and scalars are shoveled into a Pandas dataframe 
     - Some special series are generated as the expression can not be incorporated into modelflow model specifications
     - The model for fitted values is simulated in the specified timespan
     - The model for add_factors is simulated in the timespan set in MFMSA
 10. The data descriptions are extracted into a dictionary. 
    - Data descriptions for dummies, fixed values, fitted values and add_factors are derived. 
 11. Now we have a model and a dataframe with all variables which are needed.
 
The GrapWfModel instance in general keeps most of the steps so the developer can inspect the the different steps.  


## Prerequisites  

The import process requires EViews and tehrefore cannot be run on a non-windows computerthat does not have EViews 12 or later installed.  It also assumes that the python library `pyeviews` has been installed (`conda install pyeviews`).  `pyeviews` uses the EViews com interface to control eviews, allowing a python script to directly manipulate parts of the eviews model using eviews itself.  This is required only for the import process.


In [55]:
from pathlib import Path

from modelclass import model 
from modelgrabwf2 import GrabWfModel  #this is the class (part of the Modelflow Library) that will translate
                                      # the ewbvis model and equations into modelflow business logic and
                                      # import the data into pandas
model.widescreen() # Makes modelflow aware we may be running under a wider than normal screem
model.scroll_off() # tells modelflow not to scroll outputs

<IPython.core.display.Javascript object>

## Debugging the error


GrabWfModel knows some Eviews @<functions> but not all. As we can see from the line: 

 >New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3

 GrabWfModel can not make @Between go away. That is causing trouble.

The EViews help tells us:

```
@between(series, val1, val2)  Creates a dummy variable equal to 1 for observations where series is greater than or equal to val1 and less than or equal to val2.
```

In python @between(series, val1, val2) is equivalent to: 
    
 >  (1. * float(val1 <= series <= val2)))  
    
We can reproduce the logic by adding a python commmand to perform the substitution of @between in the equation with a equivalent 
python expression 


```BOL_trans = lambda text: re.sub(
    r"@BETWEEN\(\s*(\w+)\s*,\s*([^,]+)\s*,\s*([^)]+)\s*\)",
    r"(1.0*float(\2 <= \1 <= \3))",
    text
)```


In [56]:
try: 
    all_about_mda = GrabWfModel(r'wfs\bolsoln.wf1', 
                  #eviews_run_lines= mda_eviews_run_lines,
                  #country_trans    =  mda_trans,
                    make_fitted = True,        # make equatios for fitted values of stocastic equations 
                    do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                    fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                    fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                    disable_progress =True     # Better for jupyter book 
                           ) 
except Exception as e:
    print(f'Error converting model: {e}')


Reading c:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln.wf1
Assummed model name: BOL
The model: BOL is unlinked 
Writing C:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln_modelflow.wf2
Model name: BOL

Processing the model:BOL
Check for Eviews @ which are not caught in the translation
Probably errors as @ in lines:

Eviews line      :@IDENTITY BOLBFCAF2BOP_  = (1  - @DURING("1980 2021"))  * @between(reserveRatio3  , 1  / 2  , 100000)  * reserveRatio3
Original line     :@IDENTITY BOLBFCAF2BOP_  = (1  - DURING_1980_2021)  * @BETWEEN(RESERVERATIO3  , 1  / 2  , 100000)  * RESERVERATIO3
New modelflow line:BOLBFCAF2BOP_=(1-DURING_1980_2021)*@BETWEEN(RESERVERATIO3,1/2,100000)*RESERVERATIO3
Error converting model: @ in lines 


In [57]:
import re
BOL_trans = lambda text: re.sub(
    r"@between\(\s*(\w+)\s*,\s*([^,]+)\s*,\s*([^)]+)\s*\)",
    r"(1.0*float(\2 <= \1 <= \3))",
    text
)


# Usage example
text = '@between(3  , 1  / 2  , 100000)'
result = BOL_trans(text)

print(result)


(1.0*float(1  / 2   <= 3 <= 100000))


In [58]:
# some results 
print(f'{1.0*((1  / 2)   <= 30 <= 100000)=}')
print(f'{1.0*((1  / 2)   <= 30000 <= 100000)=}')
print(f'{1.0*((1  / 2)   <= 0 <= 100000)=}')

1.0*((1  / 2)   <= 30 <= 100000)=1.0
1.0*((1  / 2)   <= 30000 <= 100000)=1.0
1.0*((1  / 2)   <= 0 <= 100000)=0.0


In [59]:

BOL_trans=lambda text: re.sub(
    r"@between\(\s*(\w+)\s*,\s*([^,]+)\s*,\s*([^)]+)\s*\)",
    r"(1.0*float(\2 <= \1 <= \3))",
    text
)
all_about_bol = GrabWfModel(r'wfs\bolsoln.wf1', 
                  country_trans    =  BOL_trans,
                    make_fitted = True,        # make equatios for fitted values of stocastic equations 
                    do_add_factor_calc=True,   # Calculate the add factors which makes the stocastic equations match    
                    fit_start = 2000,          # Start of calculation of fittet model in baseline (to have some historic values) 
                    fit_end   = None,           # end of calc for fittted model, if None taken from mdmfsa options  
                    disable_progress =True     # Better for jupyter book 
                           ) 


Reading c:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln.wf1
Assummed model name: BOL
The model: BOL is unlinked 
Writing C:\modelflow manual\papers\mfbook\content\archived\howto\onboard\eviews\wfs\bolsoln_modelflow.wf2
Model name: BOL

Processing the model:BOL
Check for Eviews @ which are not caught in the translation
Default WB var_group loaded
Variable description in wf1 file read
Default WB var_description loaded self.cty='' len(var_description)=16
var_description loaded from WF len(this)=301
testmodel calculated  
Calculation of add factors for BOL calculated  


C:\modelflow2\modelflow\modelgrabwf2.py:579: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_out = self.mmodel.insertModelVar(df).pipe(self.country_df_trans).fillna(0.0)


## Check if each equation on its own result in the values provided. 
aka: residual check <br> 
If they are not pretty close, something is very wrong. 

For a more detailled display of error values use showall=1. 

In [60]:
all_about_bol.test_model(all_about_bol.start,all_about_bol.end,maxerr=100,tol=0.1,showall=1)   # tol determins the max acceptable absolute difference 

BOL calculated  

Chekking residuals for BOL 2015 to 2030

Variable with residuals above threshold
BOLNYGDPPOTLKN              , Max difference:   121.79006866 Max Pct    0.2060497541% It is number    54 in the solveorder and error number 1
FRML <IDENT> BOLNYGDPPOTLKN = BOLNYGDPTFP*(BOLLMEMPSTRL**BOLNYYWBTOTLCN_)*(BOLNEGDIKSTKKN(-1)**(1-BOLNYYWBTOTLCN_)) $

Result of equation 
          Before check  After calculation      Difference           Pct
2015 41237.2588902825   41237.2588902825    0.0000000000  0.0000000000
2016 42910.0314188969   42910.0314188969    0.0000000000  0.0000000000
2017 44488.8108181219   44488.8108181219    0.0000000000  0.0000000000
2018 46112.2953318973   46112.2953318973    0.0000000000  0.0000000000
2019 47653.1702917458   47653.1702917458    0.0000000000  0.0000000000
2020 49006.1818883145   49006.1818883145    0.0000000000  0.0000000000
2021 49885.0566274899   49885.0566274899    0.0000000000  0.0000000000
2022 51338.4205079086   51338.4205079086    0.00000

## chekout the errors

## Extract the model and the baseline
**all_about_bol** has a lot of content including. 
- .mmodel is the model instance
- .base_input is the baseline where the add factors and the fitted values are calculated 

In [65]:
mbol    = all_about_bol.mmodel       # the model instance  
baseline = all_about_bol.base_input.loc[:,sorted(mbol.allvar.keys())]#.astype('float')
reference = all_about_bol.base_input.loc[:,sorted(mbol.allvar.keys())].astype('float')

In [70]:
res = mbol(all_about_bol.base_input,2020,2030,silent=0,ldumpvar=0,solver='res',reset_options=True)
mbol.basedf = reference

BOL calculated  


In [83]:
# BOLNEGDIKSTKKN is a problem
mbol.BOLBFCAFRACGCD .eviews


@IDENTITY BOLBFCAFRACGCD  =  - BOLBFBOPTOTLCD  * (1  - BOLBFCAF2BOP_(-1))


In [77]:
baseline.BOLDEPR

1980   20.0000000000
1981   20.0000000000
1982   20.0000000000
1983   20.0000000000
1984   20.0000000000
            ...     
2046   20.0000000000
2047   20.0000000000
2048   20.0000000000
2049   20.0000000000
2050   20.0000000000
Name: BOLDEPR, Length: 71, dtype: object

## Run the model 
In order to achieve numerical stability Gauss-Seidle has to be dampened: alfa=0.7 makes sure that the solution does not explode. 
The convergence criteria is tightend a lot. 

## And the simulation result is also fine. 
Here the percent difference is displayed

In [ ]:
mbol['{cty}GGEXPCAPTCN {cty}NYGDPMKTPCN {cty}GGDBTTOTLCN {cty}BNCABFUNDCD'].dif.plot() 

## Look at a stochastic variable 
Here the equations undergo more phases 

In [ ]:
all_about_bol.all_frml_dict['bolBFCAFCAPTCD'].fprint

## Look a all the modelflow frmls
Notice after the "original" model the equations for the "fitted" values have been added. <br>
Also in the end of the listing the specification of the model which calculates the add factors if a variable is fixed. When processing the equations the ```model``` class will process this this model separately and create a model instance 
which is used to calculate add factors in case 

In [ ]:
print(mbol.equations)

In [ ]:
mbol.modeldump('test.pcim')

In [ ]:
!dir *.pcim